In [ ]:
# Panutad Sirikul
# 650510714

**20 SEP 2024**

## Dataset: Acted Emotional Speech Dynamic Database
(http://m3c.web.auth.gr/research/aesdd-speech-emotionrecognition/)

In [ ]:
!pip install tsfresh
!pip install pyts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00


* Import libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.io import wavfile
import scipy.io
import tsfresh
import matplotlib.pyplot as plt
from pyts.transformation import ShapeletTransform

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


* Read a wav file



In [ ]:
samplerate, data = wavfile.read(r'a20 (5).wav')
print(data.shape)

(194040,)


### Extract first-order statistics features

In [ ]:
mean_f = tsfresh.feature_extraction.feature_calculators.mean(data)
std_f = tsfresh.feature_extraction.feature_calculators.standard_deviation(data)
skewness_f = tsfresh.feature_extraction.feature_calculators.skewness(data)
max_f = tsfresh.feature_extraction.feature_calculators.maximum(data)
min_f = tsfresh.feature_extraction.feature_calculators.minimum(data)
first_order_fv = np.array([mean_f, std_f, skewness_f, max_f, min_f])

In [ ]:
# First Order Feature Vector
first_order_fv

array([-1.8751154e-05,  5.5784248e-02, -4.7966158e-01,  5.2403325e-01,
       -7.0794600e-01], dtype=float32)

### Extract autocorrelation features

In [ ]:
# กำหนด lag คือ จำนวน autocorrelation ที่จะดูย้อนหลังไป n steps
def autocorrelation(data, max_lag = 10):
    auto_corr_fv = []
    for lag in range(1, max_lag+1):
        tmp = tsfresh.feature_extraction.feature_calculators.autocorrelation(data, lag)
        auto_corr_fv.append(tmp)
    auto_corr_fv = np.array(auto_corr_fv)
    return auto_corr_fv
autocorr_fv = autocorrelation(data)

In [ ]:
autocorr_fv

array([ 0.84404373,  0.70912717,  0.56633067,  0.41926497,  0.2756858 ,
        0.14904208,  0.05343629, -0.00660112, -0.04120132])

In [ ]:
# สามารถใช้ร่วมกับ Feature Vector First Order ได้เช่นเอามา Concatenate
autocorr_fv = autocorrelation(data, max_lag = 15)
autocorr_fv

array([ 0.84404373,  0.70912717,  0.56633067,  0.41926497,  0.2756858 ,
        0.14904208,  0.05343629, -0.00660112, -0.04120132, -0.06543487,
       -0.08894395, -0.11468559, -0.14343274, -0.17590607])

### Extract Fourier coefficients
max_coeff จะมีค่ามากสุดเท่ากับความยาวของข้อมูลที่เรามีนะ

In [ ]:
def Fourier_coefficients(data, max_coeff = 10):
    coeff_real = []
    coeff_imag = []
    for coeff in range(0, max_coeff+1):
        tmp = tsfresh.feature_extraction.feature_calculators.fft_coefficient(data,[{"coeff":
    coeff,"attr":"real"}, {"coeff": coeff,"attr":"imag"}])
        tmp_l = list(tmp)
        coeff_real.append(tmp_l[0][1]) # บางทีอาจใช้แค่ส่วนของ real อย่างเดียวก็ได้
        coeff_imag.append(tmp_l[1][1]) # บางทีอาจใช้แค่ส่วนของ imagine อย่างเดียวก็ได้
    coeff_real = np.array(coeff_real)
    coeff_imag = np.array(coeff_imag)
    fft_fv = np.concatenate((coeff_real, coeff_imag), axis=None)
    return fft_fv

fourier_coeff_fv = Fourier_coefficients(data)

In [ ]:
fourier_coeff_fv

array([-3.63847382, -0.47646232, -0.14224474,  0.93405858, -0.47669201,
        1.10118833, -1.23326754, -0.11866813, -0.99294846,  2.71350092,
       -0.40504014,  0.        , -0.1967964 , -1.14203826, -0.57470618,
        1.65267103,  0.91536092,  0.81271282,  0.14276479,  0.05868653,
        1.08596467, -0.1209367 ])

## My Job: Choose 2 class (Folder) and Extract their Feature

### 1. Use the Acted Emotional Speech Dynamic Database
เลือก Class: disgust และ happiness

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* List all Wav files in all subfolder in the corpus

In [ ]:
import glob
import numpy as np
from scipy.io import wavfile

filenames = []
y = [] #class labels list
class_no = 0
# Updated glob pattern to include full path and file extension
for dirName in glob.glob("/content/drive/MyDrive/Feature Engineering/Acted Emotional Speech Dynamic Database/Class/*"):
    for imgFile in glob.glob(dirName + "/*.wav"):
        filenames.append(imgFile)
        y.append(class_no)
    class_no = class_no + 1

### 2. Extract feature vectors of samples on the dataset
* Retrieve each wav file and extract feature

In [ ]:
'''feature_len = 9
X = np.zeros((len(y),feature_len), dtype=float)
i = 0
for wavFile in filenames:
    #print(wavFile)
    s, data = wavfile.read(wavFile)
    X[i,:] = autocorrelation(data) # เลือก Extract Feature
    i = i + 1'''

* จะเลือกใช้ Autocorrelation รวมกับ First Order Feature Vector

In [ ]:
feature_len = 14 # ความยาวของ Feature = 9 + 5
X = np.zeros((len(y),feature_len), dtype=float)
i = 0
for wavFile in filenames:
    s, data = wavfile.read(wavFile)
    autocorr_fv = autocorrelation(data)

    mean_f = tsfresh.feature_extraction.feature_calculators.mean(data)
    std_f = tsfresh.feature_extraction.feature_calculators.standard_deviation(data)
    skewness_f = tsfresh.feature_extraction.feature_calculators.skewness(data)
    max_f = tsfresh.feature_extraction.feature_calculators.maximum(data)
    min_f = tsfresh.feature_extraction.feature_calculators.minimum(data)
    first_order_fv = np.array([mean_f, std_f, skewness_f, max_f, min_f])

    combined_fv = np.concatenate((autocorr_fv, first_order_fv), axis=None)
    X[i,:] = combined_fv
    i = i + 1

<ipython-input-36-5a449e0fcadb>:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  s, data = wavfile.read(wavFile)


### 3. Split the dataset into training and test dataset (Hint: use train_test_split method in sklearn library)

In [ ]:
# นำค่า X และ y มาทำการ train_test_split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X.shape)
print(len(y))

(241, 14)
241


In [ ]:
print(y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### 4. Construct a classifier using the training samples for identifying emotion type
* เลือกตัวแบบในการจำแนกคลาส Random Forest โดยต้องทำการ Tune ค่า Parameters ด้วย Grid Search CV ก่อน

In [ ]:
# เลือกใช้ตัวแบบ Random Forest
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# กำหนด parameter ที่ต้องการทดสอบ Grid Search CV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = RandomForestClassifier()

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

### 5. Evaluate the performance of the classifier on the test set

In [ ]:
print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Best accuracy: 0.6821862348178138


In [ ]:
best_model = RandomForestClassifier(**grid_search.best_params_)
best_model.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=10)

In [ ]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set:", accuracy)

Accuracy on test set: 0.7959183673469388


### 6. Submit your program to the assignment submission system (http://hw.cs.science.cmu.ac.th/).